<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/%EC%95%99%EC%83%81%EB%B8%94(%EB%B3%B4%ED%8C%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Labels_Safe_MSR+Juliet_no_random.csv의 사본')

In [6]:
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from datasets import load_dataset

df['code'] = df['code'].apply(lambda x: re.sub(r'/\*.*?\*/', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'//.*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include <.*?>\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include \".*?\"\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#define .*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'[\t ]+', ' ', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'\n\s*\n', '\n', x))
df.to_csv("preprocessed_data.csv", index=False)

dataset = load_dataset("csv", data_files="preprocessed_data.csv")['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from sklearn.model_selection import KFold
import torch
from torch.utils.data import DataLoader

In [8]:
# 데이터셋 로딩 및 토큰화 함수
def load_and_tokenize_data(filepath, tokenizer):
    dataset = load_dataset("csv", data_files=filepath)['train']
    tokenized_dataset = dataset.map(lambda x: tokenizer(x['code'], padding="max_length", truncation=True, max_length=512), batched=True)
    return tokenized_dataset

In [9]:
# 모델 및 토크나이저 설정
model_names = [
    "microsoft/graphcodebert-base",
    "microsoft/codebert-base-mlm",
    "microsoft/unixcoder-base"
]
models = [AutoModelForSequenceClassification.from_pretrained(name, num_labels=11) for name in model_names]
tokenizers = [AutoTokenizer.from_pretrained(name) for name in model_names]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/938k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/444k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [10]:
# K-Fold 설정 및 훈련
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
data_filepath = '/content/preprocessed_data.csv'
trained_models = []



In [15]:
for model, tokenizer in zip(models, tokenizers):
    tokenized_dataset = load_and_tokenize_data(data_filepath, tokenizer)
    for fold, (train_idx, val_idx) in enumerate(kf.split(tokenized_dataset)):
        train_subset = tokenized_dataset.select(train_idx)
        val_subset = tokenized_dataset.select(val_idx)
        training_args = TrainingArguments(
            output_dir=f'./results/fold_{fold}',
            evaluation_strategy="epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64,
            num_train_epochs=1,
            weight_decay=0.01
        )
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_subset,
            eval_dataset=val_subset
        )
        trainer.train()
        trained_models.append((model, tokenizer, val_subset))

Map:   0%|          | 0/16401 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.269600,0.614520


Epoch,Training Loss,Validation Loss
1,0.342700,0.198576


Epoch,Training Loss,Validation Loss
1,0.296500,0.173347


Epoch,Training Loss,Validation Loss
1,0.252300,0.183473


...

Epoch,Training Loss,Validation Loss
1,0.932300,0.625165


Epoch,Training Loss,Validation Loss
1,0.588300,0.468077


Epoch,Training Loss,Validation Loss
1,0.485300,0.373455


Epoch,Training Loss,Validation Loss
1,0.401100,0.330874


Epoch,Training Loss,Validation Loss
1,0.367700,0.251362


In [1]:
import torch
import numpy as np
from sklearn.metrics import f1_score

# GPU 사용 가능 여부에 따라 device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 소프트 보팅 함수 정의
def soft_voting(predictions):
    # 각 모델의 예측 확률을 평균내어 최종 예측을 결정
    ensemble_probs = torch.mean(torch.stack(predictions), dim=0)
    final_predictions = torch.argmax(ensemble_probs, dim=-1)
    return final_predictions.cpu().numpy()

def evaluate_and_voting(models_tokenizers_datasets):
    all_f1_scores = []
    for model, tokenizer, dataset in models_tokenizers_datasets:
        model.eval()
        model.to(device)
        predictions = []
        labels = []

        # 데이터셋을 DataLoader를 사용하여 반복 처리
        data_loader = DataLoader(dataset, batch_size=64, shuffle=False)
        for batch in data_loader:
            inputs = tokenizer(batch['code'], return_tensors="pt", padding="max_length", truncation=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predictions.append(probs)
            labels.extend(batch['labels'])  # 레이블 수집

        # 모든 배치에 대한 예측을 취합
        predictions = torch.cat(predictions, dim=0)
        labels = torch.tensor(labels, dtype=torch.long)  # 레이블을 텐서로 변환
        final_predictions = torch.argmax(predictions, dim=-1)
        f1_macro = f1_score(labels.numpy(), final_predictions.cpu().numpy(), average='macro')
        all_f1_scores.append(f1_macro)
    return all_f1_scores

# 평가 및 소프트 보팅 실행
f1_scores = evaluate_and_voting(trained_models)
average_f1_macro = np.mean(f1_scores)
print(f"Average F1 Macro Score: {average_f1_macro}")

NameError: name 'trained_models' is not defined